<a href="https://colab.research.google.com/github/albivaltzew/urbancode_samolet/blob/main/samolet_hack_yolo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.0/617.0 kB 6.3 MB/s eta 0:00:00


In [3]:
from sklearn.model_selection import train_test_split
import shutil
import os

In [4]:
images_path = "/content/gdrive/MyDrive/urbanhack-train/images"
labels_path = "/content/gdrive/MyDrive/urbanhack-train/labels"


In [5]:
# Read images and annotations
images = [os.path.join(images_path, x) for x in os.listdir(images_path)]
labels = [os.path.join(labels_path, x) for x in os.listdir(labels_path)]


In [6]:
images.sort()
labels.sort()

# Split the dataset into train-valid-test splits
train_images, val_images, train_labels, val_labels = train_test_split(images, labels, test_size = 0.2, random_state = 1)
val_images, test_images, val_labels, test_labels = train_test_split(val_images, val_labels, test_size = 0.5, random_state = 1)

In [7]:
!mkdir /content/data
!mkdir /content/data/train /content/data/test /content/data/val

In [8]:
!mkdir /content/data/train/images /content/data/train/labels

!mkdir /content/data/test/images /content/data/test/labels

!mkdir /content/data/val/images /content/data/val/labels

In [9]:
#Utility function to move images
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            # shutil.move(f, destination_folder)
            shutil.copy(f, destination_folder)
        except:
            print(f)
            assert False

In [10]:
train_folder = "/content/data/train"
test_folder = "/content/data/test"
val_folder = "/content/data/val"

In [11]:
# Move the splits into their folders
move_files_to_folder(train_images, train_folder+'/images')
move_files_to_folder(train_labels, train_folder+'/labels')

move_files_to_folder(val_images, val_folder+'/images')
move_files_to_folder(val_labels, val_folder+'/labels')

move_files_to_folder(test_images, test_folder+'/images')
move_files_to_folder(test_labels, test_folder+'/labels')

In [12]:
from ultralytics import YOLO

# Load a model
# model = YOLO('yolov8n.yaml')  # build a new model from YAML
# model = YOLO('/content/runs/detect/train/weights/last.pt')  # load a pretrained model (recommended for training)
# model = YOLO('yolov8n.yaml').load('yolov8n.pt')  # build from YAML and transfer weights
model = YOLO('yolov8n.pt')


100%|██████████| 6.23M/6.23M [00:00<00:00, 19.3MB/s]


In [16]:
results = model.train(data='/content/gdrive/MyDrive/urbanhack-train/data.yaml',
                      epochs=300,
                      imgsz=640,
                      patience=25,
                      save_period=1,
                      batch = 8, # 32 is to big for memory allocation, although 16
                      save_json = True)


Ultralytics YOLOv8.0.180 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/gdrive/MyDrive/urbanhack-train/data.yaml, epochs=300, patience=25, batch=8, imgsz=640, save=True, save_period=1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=True, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscrip

In [17]:
# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category

Ultralytics YOLOv8.0.180 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006233 parameters, 0 gradients
val: Scanning /content/data/val/labels.cache... 25 images, 0 backgrounds, 0 corrupt: 100%|██████████| 25/25 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:32<00:00,  8.07s/it]
                   all         25       3353      0.874      0.749      0.807      0.556
                window         25       1590      0.854      0.684      0.727       0.46
                 empty         25        787      0.942      0.802       0.89      0.665
                filled         25        976      0.826      0.761      0.804      0.542
Speed: 5.8ms preprocess, 165.9ms inference, 0.0ms loss, 57.1ms postprocess per image
Saving runs/detect/val/predictions.json...
Results saved to runs/detect/val


array([    0.46033,     0.66534,     0.54175])

In [18]:
metrics.box.map50

0.8069026850373562

проверка размерности - количество картинок совпадает с количеством меток


In [1]:
import os

# Путь к папке с изображениями
path_to_images_folder = '/content/gdrive/MyDrive/urbanhack-train/images'
# Путь к папке с разметкой в формате YOLO
path_to_yolo_folder = '/content/gdrive/MyDrive/urbanhack-train/labels'

image_extensions = ['.jpg', '.JPG']


In [11]:
# Получаем список файлов в каждой из папок, учитывая расширения файлов
images_files = os.listdir(path_to_images_folder)
yolo_files = os.listdir(path_to_yolo_folder)

252

In [13]:
# Извлекаем только имена файлов без расширения
images_without_extension = [os.path.splitext(file)[0] for file in images_files]

# Извлекаем только имена файлов без расширения
yolo_without_extension = [os.path.splitext(file)[0] for file in yolo_files]



# Находим отсутствующие файлы в папке с разметкой YOLO
missing_yolo = [file for file in images_without_extension if file not in yolo_without_extension]
missing_yolo

['IMG_6893', 'IMG_6958', 'IMG_6896', 'IMG_7521']

['IMG_6893', 'IMG_6958', 'IMG_6896', 'IMG_7521']